# GRU

Video: https://www.youtube.com/watch?v=ZMudJXhsUpY&list=PLQY2H8rRoyvzDbLUZkbudP-MFQZwNmU4S&index=6

Code: https://colab.research.google.com/github/lmoroney/dlaicourse/blob/master/TensorFlow%20In%20Practice/Course%203%20-%20NLP/Course%203%20-%20Week%204%20-%20Lesson%202%20-%20Notebook.ipynb#scrollTo=w9vH8Y59ajYL

## Setup 

In [13]:
! pip install matplotlib
! pip install tensorflowjs


In [14]:
import tensorflow as tf

from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional, Input, GRU
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam

import numpy as np 
import os

In [15]:
text_file = os.path.abspath("../exaple_text.txt")
model_file = os.path.abspath("./models/gru.h5")
js_model_dir = os.path.abspath("./models/gru_js")

## Data

In [16]:
# Datensatz laden
data = open(text_file, 'rb').read().decode(encoding='utf-8')

print(f"Inhalte des Datensatzes: \n\n{data[:250]}")

Inhalte des Datensatzes: 

Wie geht es dir?
Mir geht's gut, danke.
Was machst du heute?
Hast du gut geschlafen?
Wo wohnst du?
Ich wohne in Berlin.
Wie spät ist es?
Es ist schon ziemlich spät.
Hast du heute Zeit?
Was hast du am Wochenende vor?
Kannst du mir bitte helfen?
Was ha


In [17]:
# Corpus erstellen
tokenizer = Tokenizer()
corpus = data.lower().split("\n")
tokenizer.fit_on_texts(corpus)
total_words = len(tokenizer.word_index) + 1

print(f"Der Tokenizer kennt {total_words} Wörter")
print(f"Die Wörter werden folgenden Indexe zugenortnet: {tokenizer.word_index}")


Der Tokenizer kennt 199 Wörter
Die Wörter werden folgenden Indexe zugenortnet: {'du': 1, 'ich': 2, 'das': 3, 'hast': 4, 'habe': 5, 'was': 6, 'wie': 7, 'es': 8, 'ist': 9, 'heute': 10, 'schon': 11, 'kannst': 12, 'möchtest': 13, 'hier': 14, 'wann': 15, 'nicht': 16, 'lange': 17, 'kann': 18, 'wo': 19, 'in': 20, 'zeit': 21, 'zu': 22, 'noch': 23, 'zum': 24, 'der': 25, 'bin': 26, 'dir': 27, 'mir': 28, 'gut': 29, 'bitte': 30, 'etwas': 31, 'gehen': 32, 'muss': 33, 'war': 34, 'machst': 35, 'spät': 36, 'ziemlich': 37, 'wochenende': 38, 'frei': 39, 'wir': 40, 'uns': 41, 'mich': 42, 'schön': 43, 'den': 44, 'film': 45, 'viel': 46, 'tun': 47, 'mit': 48, 'keine': 49, 'einen': 50, 'dauert': 51, 'finde': 52, 'die': 53, 'nächste': 54, 'gibt': 55, 'komme': 56, 'bleibst': 57, 'gleich': 58, 'an': 59, 'meinen': 60, 'vergessen': 61, 'dein': 62, 'morgen': 63, 'bus': 64, 'weit': 65, 'bis': 66, 'eine': 67, 'geht': 68, "geht's": 69, 'danke': 70, 'geschlafen': 71, 'wohnst': 72, 'wohne': 73, 'berlin': 74, 'am': 75, 

In [18]:
# Datensatu generieren
input_sequences = []

# Wandelt jede Zeile in eine Liste von Wortindizes 
for line in corpus:
	token_list = tokenizer.texts_to_sequences([line])[0]

	# n-Gramm-Sequenz generieren
	for i in range(1, len(token_list)):

		n_gram_sequence = token_list[:i+1]
		input_sequences.append(n_gram_sequence)

# Alle Sequences auf gleiche Laenge bringen 
max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

# Aufteilen in Prädiktoren und Labels
xs, labels = input_sequences[:,:-1],input_sequences[:,-1]
ys = tf.keras.utils.to_categorical(labels, num_classes=total_words)

## Modell

In [19]:
# Modell Struktur 

from tensorflow.keras.layers import Input
from tensorflow.keras.models import load_model

def create_model():
    model = Sequential()
    model.add(Input(shape=(max_sequence_len - 1,)))
    model.add(Embedding(input_dim=total_words, output_dim=100, input_length=max_sequence_len - 1))
    model.add(Bidirectional(GRU(150)))
    model.add(Dense(total_words, activation='softmax'))
    
    # Optimizer und Modell-Compile
    adam = Adam(learning_rate=0.01)
    model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])
    
    return model

# Prüfen, ob das Modell bereits existiert
if os.path.exists(model_file):
    print("Modell gefunden. Lade das Modell...")
    model = load_model(model_file)  
    
else:
    print("Modell nicht gefunden. Trainiere das Modell...")
    model = create_model()  # Neues Modell erstellen
    
    # Modell trainieren
    history = model.fit(xs, ys, epochs=100, verbose=1)
    
    # Modell speichern
    model.save(model_file)
    print(f"Modell gespeichert unter: {model_file}")



Modell nicht gefunden. Trainiere das Modell...


Epoch 1/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 6s 24ms/step - accuracy: 0.0913 - loss: 5.1953
Epoch 2/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.1519 - loss: 4.3737
Epoch 3/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.2193 - loss: 3.6687
Epoch 4/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.3599 - loss: 2.6981
Epoch 5/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.4438 - loss: 2.0393
Epoch 6/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.6293 - loss: 1.3461
Epoch 7/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.6531 - loss: 1.1248
Epoch 8/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.6586 - loss: 1.0132
Epoch 9/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.6599 - loss: 0.9901
Epoch 10/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.7063 - loss: 0.8034
Epoch 11/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.7096 - loss: 0.8101
Epoch 12/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step

Modell gespeichert unter: /workspace/silvan/algorithem/next_word_prediciton/models/gru.h5


## Verwendung 

### Text generieren

In [20]:
seed_text = "Hallo ich bin "
next_words = 1
  
for _ in range(next_words):
	token_list = tokenizer.texts_to_sequences([seed_text])[0]
	token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
	predicted = np.argmax(model.predict(token_list), axis=-1)
	output_word = ""
	for word, index in tokenizer.word_index.items():
		if index == predicted:
			output_word = word
			break
	seed_text += " " + output_word
print(seed_text)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 639ms/step
Hallo ich bin  sehr


### Nextes Word vorhesagen  

In [21]:
import numpy as np

seed_text = "Wie geht"
top_k = 10  # Anzahl der Top-Wörter

# Tokenisierung und Padding des Eingabetexts
token_list = tokenizer.texts_to_sequences([seed_text])[0]
token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')

# Vorhersagen berechnen
predictions = model.predict(token_list, verbose=0)[0]  # Softmax-Ausgabe

# Top-10 wahrscheinlichste Wörter finden
top_indices = np.argsort(predictions)[-top_k:][::-1]  #Sortiere absteigend

# Wörter und Wahrscheinlichkeiten sammeln
top_words = [(word, predictions[index]) for word, index in tokenizer.word_index.items() if index in top_indices]

# Sortiere die Wörter nach Wahrscheinlichkeit in absteigender Reihenfolge
top_words = sorted(top_words, key=lambda x: x[1], reverse=True)

# Ausgabe der Top-10 Wörter und Wahrscheinlichkeiten
print(f"Top {top_k} nächste Wörter für '{seed_text}':")
for word, prob in top_words:
    print(f"{word}: {(prob * 100):.4f} %")


Top 10 nächste Wörter für 'Wie geht':
es: 99.4833 %
die: 0.1147 %
meinen: 0.0794 %
dir: 0.0414 %
mit: 0.0366 %
ich: 0.0306 %
das: 0.0280 %
ist: 0.0258 %
du: 0.0221 %
dein: 0.0183 %


## Konvertierung 

In [22]:
! tensorflowjs_converter --input_format=keras {model_file} {js_model_dir}

2024-10-16 14:35:20.890706: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-16 14:35:21.247644: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-16 14:35:21.334834: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-16 14:35:28.536746: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
failed to lookup keras version from the file,
    this is likely a weight only file
